In [1]:
import sqlite3
import psycopg2
import pandas as pd
import numpy as np
import math
from DataBase_functions import *
from DataBase_functions_testing import *

#Google sheed API
import gspread
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

<p style="font-size: 30px; text-align: center; color: #20a7e5;">Creating data-frame</p>

<p style="font-size: 25px; color: #208ee5">Registro actividad todos</p>

In [2]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "12qLRAp7C9qzcoWkyUTJAVMhRHMFfwYec094J8CU2t7c" #Registro actividad todos: 12qLRAp7C9qzcoWkyUTJAVMhRHMFfwYec094J8CU2t7c

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #Dataframe con los datos de todos los meses
    df = pd.DataFrame(columns=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]) 
    meses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
    for sheet in sheets:
        sheet_name = sheet.title 
        if sheet_name in meses:
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]))
            df = pd.concat([df, df_aux], ignore_index=True)
        else: 
            print(sheet_name, "is not a month")
            pass
    #df = df.drop_duplicates(subset='Código', keep='first')
    print(df.shape, "\n", df.columns)

    #Dataframe with 2024 sheet data
    sheet_2024 = spreadsheet.worksheet("2024")
    df_2024 = pd.DataFrame(sheet_2024.get_all_records(expected_headers=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]))
    #df_2024 = df_2024.drop_duplicates(subset='Código', keep='first')
    print(df_2024.shape, "\n", df_2024.columns)

    #Concatenate both dataframes
    df = pd.concat([df, df_2024], ignore_index=True)
    df = df.drop_duplicates(subset="Código", keep="first")
    print(df.shape)

    #Dataframe validation
    df.rename(columns={"Validada": "validada", "Responsable": "responsable", "Gestor DDBB": "DDBB", "Geografía": "geography", "Petición": "petition_code", "Feature": "feature", "SDATOOL": "sdatool", "Código": "DQDP_code", "Gestor BBDD": "DDBB", "Horas": "duration_time", "Comentarios": "description", "Fecha de alta": "fecha_in", "Fecha de fin": "fecha_out"}, inplace=True)

    df["fecha_in"] = df["fecha_in"].astype(str)
    df["Fecha incurrida"] = df["Fecha incurrida"].astype(str)
    df["fecha_out"] = df["fecha_out"].astype(str)
    df["UUAA"] = df["UUAA"].astype(str)
    df["DQDP_code"] = df["DQDP_code"].astype(str)
    df["sdatool"] = df["sdatool"].astype(str)
    df["feature"] = df["feature"].astype(str)
    df["geography"] = df["geography"].astype(str)
    df["DDBB"] = df["DDBB"].astype(str)
    df["Ámbito"] = df["Ámbito"].astype(str)
    df["responsable"] = df["responsable"].astype(str)
    df["validada"] = df["validada"].astype(str)
    df["duration_time"] = pd.to_numeric(df["duration_time"], errors="coerce")
    df["description"] = df["description"].astype(str)
    df["petition_code"] = df["petition_code"].astype(str)

    df["version_date"] = "Nan"
    df["version"] = "Nan"
    df["petition_arq"] = "Nan"
    df["dev_master"] = "Nan"
    df["estado"] = "Nan"

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['2024 totales - Peticiones', '2024 totales - Peticiones - Pro', '2024', 'Gráficos Semanales - Procesos', '2024 - Procesos', 'Abril', 'Mayo', 'Julio', 'Junio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre', 'Responsables']
2024 totales - Peticiones is not a month
2024 totales - Peticiones - Pro is not a month
2024 is not a month
Gráficos Semanales - Procesos is not a month
2024 - Procesos is not a month
Responsables is not a month
(4371, 18) 
 Index(['Fecha de alta', 'Fecha incurrida', 'Fecha de fin', 'UUAA', 'Código',
       'SDATOOL', 'Feature', 'Petición', 'Geografía', 'Gestor BBDD', 'Ámbito',
       'Responsable', 'Validada', 'Horas', 'Comentarios', 'Estado', 'Días',
       ''],
      dtype='object')
(1974, 18) 
 Index(['Fecha de alta', 'Fecha incurrida', 'Fecha de fin', 'UUAA', 'Código',
       'SDATOOL', 'Feature', 'Petición', 'Geografía', 'Gestor BBDD', 'Ámbito',
       'Responsable', 'Validada', 'Horas', 'Comentarios', 'Estado', 'Días',
       ''

<p style="font-size: 25px; color: #208ee5; text-align: center;">Peticiones globales Oracle, Elastic Search y Mongo DB</p>

<p style="font-size: 25px; color: #208cc5;;">Oracle Physics</p>

In [3]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1PZKCpybEechpbwP-H3khhUihHqwUqgbi_KkM6iCxhuE" #Peticiones globales Oracle: "1PZKCpybEechpbwP-H3khhUihHqwUqgbi_KkM6iCxhuE"

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    df_OG = pd.DataFrame(columns=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"])

    for sheet in sheets: 
        try: 
            sheet_name = sheet.title
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"]))
            df_OG = pd.concat([df_OG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_OG["DDBB"] = "Oracle Physics"
    df_OG["geography"] = "Global"

    print(df_OG.shape)

    df_OG["UUAA"] = df_OG["UUAA"].astype(str)
    df_OG["Petición Jira"] = df_OG["Petición Jira"].astype(str)
    df_OG["Versión Repo"] = df_OG["Versión Repo"].astype(str)
    df_OG["Petición ARQ"] = df_OG["Petición ARQ"].astype(str)
    df_OG["Fecha cierre"] = df_OG["Fecha cierre"].astype(str)
    df_OG["Comentarios"] = df_OG["Comentarios"].astype(str)
    df_OG["Path"] = df_OG["Path"].astype(str)
    df_OG["DDBB"] = df_OG["DDBB"].astype(str)
    df_OG["geography"] = df_OG["geography"].astype(str)

    df_OG.rename(columns={"Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['APIC', 'ATAU', 'B1DS', 'B43T', 'BJP9', 'CCUS', 'DAMS', 'DMTO', 'EGRP', 'EI4M', 'ESCD', 'FZHI', 'GDEL', 'GHDC', 'GJSX', 'GSIN', 'GUAR', 'H9IY', 'HFPP', 'HLCY', 'HPGR', 'IMPT', 'J6G7', 'K5VI', 'KAPI', 'KARC', 'KBGE', 'KBTQ', 'KCCX', 'KCDO', 'KCMC', 'KCMS', 'KCNC', 'KCNS', 'KCOG', 'KCOL', 'KCSN', 'KCSP', 'KDCL', 'KDFH', 'KDMG', 'KERF', 'KESJ', 'KFGC', 'KFRE', 'KFUL', 'KGCC', 'KGCI', 'KGTP', 'KIFV', 'KIVS', 'KLIM', 'KLNE', 'KMET', 'KMIC', 'KMOL', 'KMRK', 'KMYC', 'KNWD', 'KOMN', 'KONB', 'KPAG', 'KPAD', 'KPAY', 'KPDA', 'KPDR', 'KPFE', 'KPFM', 'KPIG', 'KPPS', 'KPST', 'KRBA', 'KRCE', 'KSAG', 'KSAN', 'KSDD', 'KSPH', 'KSRO', 'KUSU', 'KVCS', 'KXSG', 'KYSZ', 'KYOP', 'KYUB', 'MCRR', 'MFCG ', 'MK0W', 'MDDM', 'NI0C', 'O43T', 'PGVC', 'PIVU', 'PMSC', 'PMSV', 'P6ES', 'QWJR', 'QWPJ', 'RLR7', 'SWP3', 'S47B', 'UGX2', 'V34F', 'VAMO', 'W1BD', 'XBOB', 'ZGR7', 'XMBR', 'XR6Z', 'ZDNM', 'ZNXD']
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
(947, 10)


<p style="font-size: 25px; color: #208cc5;;">Elastic Search</p>

In [4]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1PXFKrrJVs12XpmgV5DmnBvjxENQ3hobwLs-bdDf8OIE" #Peticiones globales Mongo: "1PXFKrrJVs12XpmgV5DmnBvjxENQ3hobwLs-bdDf8OIE"

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    df_EG = pd.DataFrame(columns=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"])

    for sheet in sheets: 
        try: 
            sheet_name = sheet.title
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"]))
            df_EG = pd.concat([df_EG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_EG["DDBB"] = "Oracle Physics"
    df_EG["geography"] = "Global"

    #df = df_OG.drop_duplicates(subset='Código', keep='first')
    print(df_EG.shape)

    df_EG["UUAA"] = df_EG["UUAA"].astype(str)
    df_EG["Petición Jira"] = df_EG["Petición Jira"].astype(str)
    df_EG["Versión Repo"] = df_EG["Versión Repo"].astype(str)
    df_EG["Petición ARQ"] = df_EG["Petición ARQ"].astype(str)
    df_EG["Fecha cierre"] = df_EG["Fecha cierre"].astype(str)
    df_EG["Comentarios"] = df_EG["Comentarios"].astype(str)
    df_EG["Path"] = df_EG["Path"].astype(str)
    df_EG["DDBB"] = df_EG["DDBB"].astype(str)
    df_EG["geography"] = df_EG["geography"].astype(str)

    df_EG.rename(columns={"Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['ATAU', 'F30V', 'EI4M', 'GHDC', 'GUAR', 'H9IY', 'IMPT', 'J6G7', 'JBSY', 'KAGE', 'KAPI', 'KBTQ', 'KCNS', 'KHR1', 'KIFV', 'KLNE', 'KMOL', 'KREC', 'KTIN', 'KTRA', 'KTR2', 'KXSG', 'KYGU', 'KYOP', 'L1WI', 'NI0C', 'O43T', 'OPEI', 'PGVC', 'S47B', 'XBMR', 'XPI2', 'YM3X', 'ZNXD']
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
(224, 10)


<p style="font-size: 25px; color: #208cc5;">Mongo DB</p>

In [5]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1-mO_2AFa0J4VBlONUFPuQ7IfTRNxBb3BbPyEg_D-FPs" #Peticiones globales Mongo: "1-mO_2AFa0J4VBlONUFPuQ7IfTRNxBb3BbPyEg_D-FPs"

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    df_MG = pd.DataFrame(columns=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"])

    for sheet in sheets: 
        try: 
            sheet_name = sheet.title
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"]))
            df_MG = pd.concat([df_MG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_MG["DDBB"] = "Oracle Physics"
    df_MG["geography"] = "Global"
    
    print(df_MG.shape)

    df_MG["UUAA"] = df_MG["UUAA"].astype(str)
    df_MG["Petición Jira"] = df_MG["Petición Jira"].astype(str)
    df_MG["Versión Repo"] = df_MG["Versión Repo"].astype(str)
    df_MG["Petición ARQ"] = df_MG["Petición ARQ"].astype(str)
    df_MG["Fecha cierre"] = df_MG["Fecha cierre"].astype(str)
    df_MG["Comentarios"] = df_MG["Comentarios"].astype(str)
    df_MG["Path"] = df_MG["Path"].astype(str)
    df_MG["DDBB"] = df_MG["DDBB"].astype(str)
    df_MG["geography"] = df_MG["geography"].astype(str)

    df_MG.rename(columns={"Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)
    
except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['KSKR', 'KPDR', 'F30V', 'KAPI', 'H9IY', 'KPAD', 'CHM8', 'XMBR', 'KCOG', 'KONL', 'KNWD', 'KIFV', 'KIFD', 'KPFM', 'KHR1', 'KARC', 'KATT']
(77, 9)


<p style="font-size: 25px; color: #20cff5;">All in one Data Frame</p>

In [6]:
df_g = pd.DataFrame(columns=list(df.columns))
df_g = pd.concat([df_g, df_OG, df_EG, df_MG], ignore_index=True)

columns_to_use = ["fecha_in", "fecha_out", "UUAA", "DQDP_code", "sdatool", "feature", "geography", "DDBB", "Ámbito", "responsable", "validada", "description", "version_date", "version", "petition_arq", "estado", "petition_code", "duration_time", "dev_master"]

dataframes = [df, df_g]
for column in columns_to_use: 
    for i in dataframes:
        i[column] = i[column].astype(str)

df["duration_time"] = pd.to_numeric(df["duration_time"], errors="coerce")
df["dev_master"] = "None"

df_g["duration_time"] = pd.to_numeric(df_g["duration_time"], errors="coerce")
df_g["dev_master"] = "None"

####################################################################################################
dataframes_using = [df, df_g]
for i in dataframes_using:
    for row in i.iterrows(): 
        for key, value in dict(row[1]).items():
            if value == "Nan" or value == None or value == "" or value == "nan" or value == "None" or value == "none" or value == "NONE" or value == np.nan: 
                if key == "fecha_in" or key == "fecha_out" or key == "Fecha incurrida" or key == "UUAA" or key == "DQDP_code" or key == "sdatool" or key == "feature" or key == "geography" or key == "DDBB" or key == "Ámbito" or key == "responsable" or key == "validada" or key == "description" or key == "version_date" or key == "version" or key == "petition_arq" or key == "estado" or key == "petition_code" or key == "dev_master":
                    i.at[row[0], key] = "None"
                if key == "duration_time":
                    i.at[row[0], key] = np.nan
                else: 
                    pass

df_g.drop_duplicates(subset=["petition_code"], keep="first", inplace=True)
df_g = df_g[df_g["petition_code"] != "None"]

#df = pd.concat([df, df_g], ignore_index=True)

In [7]:
df_definitive = pd.DataFrame(columns=columns_to_use)
petitions = set(df_g["petition_code"]).union(set(df["petition_code"]))
print(len(petitions), df_definitive.columns)
petitions

2362 Index(['fecha_in', 'fecha_out', 'UUAA', 'DQDP_code', 'sdatool', 'feature',
       'geography', 'DDBB', 'Ámbito', 'responsable', 'validada', 'description',
       'version_date', 'version', 'petition_arq', 'estado', 'petition_code',
       'duration_time', 'dev_master'],
      dtype='object')


{'OTRSSD-16760',
 'OTRSSD-13691',
 'FIFN_20240704165726',
 'OTRSSD-14529',
 'OTRSSD-15594',
 'SGS-13308',
 'SGS-17450',
 'OTRSSD-14557',
 'OTRSSD-16969',
 'REQ000101384824',
 'OTRSSD-17265',
 'OTRSSD-7084',
 'OTRSSD-9130',
 'DSFI_20241021125915',
 'OTRSSD-14332',
 'SOTRSSD-14094',
 'OTRSSD-16561',
 'OTRSSD-16040',
 'OTRSSD-4925',
 'OTRSSD-16343',
 'OTRSSD-13120',
 'KDHJ_20240529074933',
 'OTRSSD-11717',
 'OTRSSD-12535',
 'OTRSSD-17168',
 'OTRSSD-16562',
 'OTRSSD-17522',
 'OTRSSD-7029',
 'KXBC_20240422131618',
 'OTRSSD-7840',
 'OTRSSD-16039',
 'TVPV_20240321125108',
 'SGS-9132',
 'OTRSSD-18295',
 'OTRSSD-16333',
 'OTRSSD-17646',
 'OTRSSD-16248',
 'OTRSSD-14593',
 'IDG000250225',
 'REQ000101364306',
 'SGS-13651',
 'OTRSSD-17548',
 'OTRSSD-17242',
 'REQ000100974944',
 'OTRSSD-9281',
 'OTRSSD-14464',
 'KPSA_20240521213125',
 'OTRSSD-14967',
 'OTRSSD-5867',
 'OTRSSD-14080',
 'SGS-14321',
 'OTRSSD-9934',
 'OTRSSD-7904',
 'OTRSSD-17990',
 'RRAS_20240404133307',
 'OTRSSD-8673',
 'OTRSSD-12660'

In [8]:
for petition in petitions: 
    dic_aux = {i: [] for i in columns_to_use}

    for column in columns_to_use: 
        df_filtered = df[df["petition_code"] == petition]
        df_g_filtered = df_g[df_g["petition_code"] == petition]
        
        if not df_filtered.empty: 
            df_value = df_filtered[column].values[0]
        else: 
            df_value = "None" if column != "duration_time" else None

        if not df_g_filtered.empty: 
            df_g_value = df_g_filtered[column].values[0]
        else: 
            df_g_value = "None" if column != "duration_time" else None
        

        if df_value != "None" and df_value is not None: 
            dic_aux[column].append(df_value)
        elif df_g_value != "None" and df_g_value is not None: 
            dic_aux[column].append(df_g_value)
        else:
            dic_aux[column].append(df_value) #None o "None"

    df_aux = pd.DataFrame(dic_aux)
    df_definitive = pd.concat([df_definitive, df_aux], ignore_index=True)

C:\Users\matia\AppData\Local\Temp\ipykernel_20804\2068147553.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_definitive = pd.concat([df_definitive, df_aux], ignore_index=True)


In [9]:
dfs = [df, df_g, df_definitive]
df_none_count = {i: 0 for i in list(dfs[2].columns)}
for row in dfs[2].iterrows(): 
    for key, value in dict(row[1]).items():
        if value == "None" or value == None or (isinstance(value, float) and np.isnan(value)):
            df_none_count[key] += 1
        else: 
            pass
print(dfs[0].shape, dfs[1].shape, dfs[2].shape)
df_none_count

(1739, 23) (1134, 24) (2362, 19)


{'fecha_in': 970,
 'fecha_out': 306,
 'UUAA': 2,
 'DQDP_code': 711,
 'sdatool': 796,
 'feature': 771,
 'geography': 205,
 'DDBB': 129,
 'Ámbito': 879,
 'responsable': 1287,
 'validada': 1498,
 'description': 1874,
 'version_date': 2362,
 'version': 1231,
 'petition_arq': 1278,
 'estado': 2362,
 'petition_code': 1,
 'duration_time': 769,
 'dev_master': 2362}

In [10]:
df_definitive

,fecha_in,fecha_out,UUAA,DQDP_code,sdatool,feature,geography,DDBB,Ámbito,responsable,validada,description,version_date,version,petition_arq,estado,petition_code,duration_time,dev_master
0,20/08/2024,20/08/2024,KGIN,DQ-DP-006821,45148,RENOV-23,España,ORACLE Physics,NextGEN,Mario,Si,None,None,None,None,None,OTRSSD-16760,0.75,None
1,12/03/2024,12/03/2024,EJTM,DQ-DP-005884,41253,-,España,ORACLE Physics,NextGEN,Mario,None,None,None,None,None,None,OTRSSD-13691,2.00,None
2,10/07/2024,None,FIFN,DQ-DP-006618,11948,FACTORING-113,España,DB2 Host,HOST,Mario,Si,En curso,None,None,None,None,FIFN_20240704165726,1.00,None
3,None,None,KYFE,DQ-DP-006110,45317,ARCCOMMUNI-2929,None,None,None,Mario,Si,None,None,None,None,None,OTRSSD-14529,0.50,None
4,30/05/2024,30/05/2024,KPAY,DQ-DP-006399,45538,SWP2PA-8,Global,ORACLE Physics,NextGEN,Mario,Si,None,None,12,Helix: 95285,None,OTRSSD-15594,1.00,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2357,20/02/2024,21/02/2024,CXCX,DQ-DP-005749,39718,-,España,DB2 HOST,HOST,None,None,None,None,None,None,None,CXCX_20240219130214,3.00,None
2358,16/12/2024,None,KRBA,DQ-DP-007511,---,---,Global,Oracle Physics,NextGEN,None,None,4.7.33 BKRBA001 v17 (aprobada por Alfonso) = B...,None,None,None,None,WO0000101348011,NaN,None
2359,19/12/2024,26/12/2024,GDEL,DQ-DP-007517,44854,WSONEFUTUR-7,Global,Oracle Physics,NextGEN,None,None,3.1.3 master v10 --> 3.4.3 dev v4 (posible v11...,None,v3 dev -> posible v11 master,WO0000101345984,None,OTRSSD-18489,2.00,None
2360,None,15/02/2023,KCMC,None,None,None,Global,Oracle Physics,None,None,None,None,None,4,DATASD-59493,None,OTRSSD-5901,NaN,None


---
---

In [11]:
'''
for row in df_definitive.iterrows():
    insert_data_testing(dict(row[1]))
'''

'\nfor row in df_definitive.iterrows():\n    insert_data_testing(dict(row[1]))\n'

<p style="font-size: 40px; color: grey;">Testing</p>

In [12]:
conn1 = sqlite3.connect("BBVA_testing.db")
cursor1 = conn1.cursor()
cursor1.execute("PRAGMA foreign_keys = ON") #In sqlite3 foreign keys are disabled by default

#######
cursor1.execute("SELECT UUAA FROM UUAA")
all_uuaa = [x[0] for x in cursor1.fetchall()]

#######
conn1.commit()
cursor1.close()
conn1.close()